In [3]:
import pandas as pd
# modified from https://stackabuse.com/implementing-pca-in-python-with-scikit-learn/
dataset = pd.read_csv("ChemOptCleaned3.csv", header=0, index_col=0)
dataset.head()


,bact,apol,ASA,ASA+,ASA-,ASA_H,ASA_P
mol,,,,,,,
TLcf,high,373.22794,3192.7236,2299.1519,893.5719,1840.1213,1352.6024
THIS5,high,570.93018,4632.4492,0.0000,0.0000,4632.4492,0.0000
Lcf,low,237.45761,2081.5366,0.0000,0.0000,2081.5366,0.0000
HIS5,low,450.00461,3726.2500,0.0000,0.0000,3726.2500,0.0000


In [4]:
X = dataset.drop('bact', 1)
y = dataset['bact']
# The script above stores the feature sets into the X variable and the series of 
# corresponding labels in to the y variable.

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)


/usr/lib64/python3.7/site-packages/sklearn/model_selection/_search.py:16: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, namedtuple, defaultdict, Sequence


As mentioned earlier, PCA performs best with a normalized feature set. We will perform standard scalar normalization to normalize our feature set. To do this, execute the following code:

In [5]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Applying PCA

It is only a matter of three lines of code to perform PCA using Python's Scikit-Learn library. The PCA class is used for this purpose. PCA depends only upon the feature set and not the label data. Therefore, PCA can be considered as an unsupervised machine learning technique.

Performing PCA using Scikit-Learn is a two-step process:

    Initialize the PCA class by passing the number of components to the constructor.
    Call the fit and then transform methods by passing the feature set to these methods. The transform method returns the specified number of principal components.

Take a look at the following code:

In [6]:
from sklearn.decomposition import PCA

pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In the code above, we create a PCA object named pca. We did not specify the number of components in the constructor. Hence, all four of the features in the feature set will be returned for both the training and test sets.

The PCA class contains explained_variance_ratio_ which returns the variance caused by each of the principal components. Execute the following line of code to find the "explained variance ratio".

In [7]:
explained_variance = pca.explained_variance_ratio_

The explained_variance variable is now a float type array which contains variance ratios for each principal component. The values for the explained_variance variable looks like this:
0.722265
0.239748
0.0333812
0.0046056

It can be seen that first principal component is responsible for 72.22% variance. Similarly, the second principal component causes 23.9% variance in the dataset. Collectively we can say that (72.22 + 23.9) 96.21% percent of the classification information contained in the feature set is captured by the first two principal components.

Let's first try to use 1 principal component to train our algorithm. To do so, execute the following code:

In [8]:
from sklearn.decomposition import PCA

pca = PCA(n_components=1)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

The rest of the process is straight forward.

Training and Making Predictions

In this case we'll use random forest classification for making the predictions.

In [9]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

/usr/lib64/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [10]:
# Performance Evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy' + str(accuracy_score(y_test, y_pred)))

[[0 0]
 [1 0]]
Accuracy0.0


In [11]:
y_pred

array(['high'], dtype=object)

In [12]:
y_test

mol
Lcf    low
Name: bact, dtype: object

It can be seen from the output that with only one feature, the random forest algorithm is able to correctly predict 28 out of 30 instances, resulting in 93.33% accuracy.
Results with 2 and 3 Principal Components

Now let's try to evaluate classification performance of the random forest algorithm with 2 principal components. Update this piece of code:

In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

ValueError: n_components=2 must be between 0 and n_features=1 with svd_solver='full'